check how many of the generated headlines are compliant with:

- the original headline
- the original article

using the provided classifiers

## Download models

In [1]:
!wget -O h-a-classifier.h5 https://unipiit-my.sharepoint.com/:u:/g/personal/l_demattei_studenti_unipi_it/Ee6SwfKkLzFJk-_1ho3LXFUB0fZlADKGdod42qM0dmvGZA?download=1

--2020-08-28 19:02:00--  https://unipiit-my.sharepoint.com/:u:/g/personal/l_demattei_studenti_unipi_it/Ee6SwfKkLzFJk-_1ho3LXFUB0fZlADKGdod42qM0dmvGZA?download=1
Resolving unipiit-my.sharepoint.com (unipiit-my.sharepoint.com)... 13.107.136.9
Connecting to unipiit-my.sharepoint.com (unipiit-my.sharepoint.com)|13.107.136.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/l_demattei_studenti_unipi_it/Documents/h-a-bm-classifiers.h5?&originalPath=aHR0cHM6Ly91bmlwaWl0LW15LnNoYXJlcG9pbnQuY29tLzp1Oi9nL3BlcnNvbmFsL2xfZGVtYXR0ZWlfc3R1ZGVudGlfdW5pcGlfaXQvRWU2U3dmS2tMekZKay1fMWhvM0xYRlVCMGZabEFES0dkb2Q0MnFNMGRtdkdaQT9ydGltZT16SFc1LTRSTDJFZw [following]
--2020-08-28 19:02:01--  https://unipiit-my.sharepoint.com/personal/l_demattei_studenti_unipi_it/Documents/h-a-bm-classifiers.h5?&originalPath=aHR0cHM6Ly91bmlwaWl0LW15LnNoYXJlcG9pbnQuY29tLzp1Oi9nL3BlcnNvbmFsL2xfZGVtYXR0ZWlfc3R1ZGVudGlfdW5pcGlfaXQvRWU2U3dmS2tMekZKay1fMWhvM0xYRlVCMGZabEFES0dkb2Q0MnFNMGRtdkdaQT9y

In [2]:
!wget -O h-h-classifier.h5 https://unipiit-my.sharepoint.com/:u:/g/personal/l_demattei_studenti_unipi_it/EZhDwNJSfz1No5WT9t5hOjMB7zrXdJBBu2dGEZCxElIhqA?download=1

--2020-08-28 19:08:53--  https://unipiit-my.sharepoint.com/:u:/g/personal/l_demattei_studenti_unipi_it/EZhDwNJSfz1No5WT9t5hOjMB7zrXdJBBu2dGEZCxElIhqA?download=1
Resolving unipiit-my.sharepoint.com (unipiit-my.sharepoint.com)... 13.107.136.9
Connecting to unipiit-my.sharepoint.com (unipiit-my.sharepoint.com)|13.107.136.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/l_demattei_studenti_unipi_it/Documents/h-h-bm-classifiers.h5?&originalPath=aHR0cHM6Ly91bmlwaWl0LW15LnNoYXJlcG9pbnQuY29tLzp1Oi9nL3BlcnNvbmFsL2xfZGVtYXR0ZWlfc3R1ZGVudGlfdW5pcGlfaXQvRVpoRHdOSlNmejFObzVXVDl0NWhPak1CN3pyWGRKQkJ1MmRHRVpDeEVsSWhxQT9ydGltZT10MkhzOFlWTDJFZw [following]
--2020-08-28 19:08:54--  https://unipiit-my.sharepoint.com/personal/l_demattei_studenti_unipi_it/Documents/h-h-bm-classifiers.h5?&originalPath=aHR0cHM6Ly91bmlwaWl0LW15LnNoYXJlcG9pbnQuY29tLzp1Oi9nL3BlcnNvbmFsL2xfZGVtYXR0ZWlfc3R1ZGVudGlfdW5pcGlfaXQvRVpoRHdOSlNmejFObzVXVDl0NWhPak1CN3pyWGRKQkJ1MmRHRVpDeEVsSWhxQT9y

In [1]:
import pandas as pd
import numpy as np
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer

In [2]:
dataset = pd.DataFrame(columns=['rep_original_headlines', 'rep_original_fulltext', 'gio_generated_headlines'])
dataset['rep_original_headlines'] = open('../rep/val_rep_h.txt', 'r').read().splitlines()
dataset['rep_original_fulltext'] = open('../rep/val_rep_ft.txt', 'r').read().splitlines()
dataset['gio_generated_headlines'] = open('../pred.txt', 'r').read().splitlines()

In [3]:
dataset

,rep_original_headlines,rep_original_fulltext,gio_generated_headlines
0,"Decreto sicurezza, denunciati per accattonaggi...",BRESCIA - Per la prima volta nel Bresciano vie...,Bresciano nel Bresciano Gesù: contestato
1,"Sicilia, i siriani scampati al naufragio riabb...","SANT'ANGELO MUXARO (Agrigento) - ""Karim"". È un...","Agrigento, strozzata strozzata dall'emozione,"
2,"Stati Uniti, uragano Florence: polemica sulle ...","WASHINGTON. ""Florence è molto pericoloso però ...",Ecco i politici del Nord
3,Se il pentastellato parla da Le Pen,"Populismo è ormai una parola ""malata"", un cont...",Anis Amri a Grillo dopo la morte di Grillo
4,La soluzione dello schema del 2 Dicembre 2017,Torna al cruciverba Torna all'indice della rub...,Il rubrica della fiamma Scrivi della chiesa al...
...,...,...,...
12735,"Thailandia, destituita la premier per abuso di...",BANGKOK - Ciò che non era riuscito all'opposiz...,"sorella del Nord, progressi la sorella del"
12736,"Geronimo Stilton: ""Su carta o in digitale conq...",Il re italiano indiscusso dei bestseller è un ...,Il commento dei bestseller è un topo
12737,"""BlackBerry pensa a lanciare uno smartphone co...","SE CONFERMATA, la notizia avrebbe del clamoros...",BlackBerry sceglie la linea: recensione nel 2009
12738,"Nardella: ""Cuperlo ammetta la sconfitta. La sc...","FIRENZE - ""Il giorno dopo le elezioni hanno vi...",D'Alema e i cuperliani Cuperlo di Renzi


In [4]:
# Defining some key variables that will be used later on in the training
HH_MAX_LEN = 64
HH_VALID_BATCH_SIZE = 128
HA_MAX_LEN = 512
HA_VALID_BATCH_SIZE = 8
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

In [5]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, field):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.field = field
        
    def __getitem__(self, index):
        if self.field == 'fulltext':
            rep_orginal = str(self.data.rep_original_fulltext[index])
        else:
            rep_orginal = str(self.data.rep_original_headlines[index])
        rep_orginal = " ".join(rep_orginal.split())
        
        
        gio_headline_generated = str(self.data.gio_generated_headlines[index])
        gio_headline_generated = " ".join(gio_headline_generated.split())
        
        inputs = self.tokenizer.encode_plus(
            rep_orginal,
            gio_headline_generated,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        #print(inputs)
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        #print(title, text)
        #print(ids)

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
        } 
    
    def __len__(self):
        return self.len
    
# Creating the dataset and dataloader for the neural networkkenizer, MAX_LEN)
hh_testing_set = Triage(dataset.reset_index(drop=True), tokenizer, HH_MAX_LEN, "headline")
hh_test_params = {'batch_size': HH_VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }
hh_testing_loader = DataLoader(hh_testing_set, **hh_test_params)

ha_testing_set = Triage(dataset.reset_index(drop=True), tokenizer, HA_MAX_LEN, "fulltext")
ha_test_params = {'batch_size': HA_VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }
ha_testing_loader = DataLoader(ha_testing_set, **ha_test_params)

In [6]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained('bert-base-multilingual-cased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 1)
    
    def forward(self, ids, mask):
        output_1= self.l1(ids, mask)
        output_2 = self.l2(output_1[0])
        output = self.l3(output_2)
        return output

device = 'cuda:0'
model = BERTClass()
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   

In [7]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0
    y_target = []
    y_predicted = []
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            total += outputs.size(0)
            n_correct += big_idx.sum().item()
            
    return ((total-n_correct)*100.0)/total

In [8]:
HH_MODEL_PATH = "h-h-classifier.h5"
model.load_state_dict(torch.load(HH_MODEL_PATH, map_location=device))

<All keys matched successfully>

In [9]:
hh_accuracy = valid(model, hh_testing_loader)

In [10]:
HH_MODEL_PATH = "h-a-classifier.h5"
model.load_state_dict(torch.load(HH_MODEL_PATH, map_location=device))

<All keys matched successfully>

In [11]:
ha_accuracy = valid(model, ha_testing_loader)

In [12]:
print(f"Headlines-Headlines match accuracy {hh_accuracy}")
print(f"Headlines-Article match accuracy {ha_accuracy}")

Headlines-Headlines match accuracy 59.94505494505494
Headlines-Article match accuracy 87.2370486656201
